## Giới thiệu và Nhập Thư viện

In [21]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# gọi nltk_utils.py và model.py 
from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

## Tải và Xử lý Sơ bộ Dữ liệu Ý định (Intents)

In [22]:
with open('intents.json', encoding='utf-8') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# Lặp qua mỗi câu trong các mẫu ý định
for intent in intents['intents']:
    tag = intent['tag']
    # Thêm vào danh sách tag
    tags.append(tag)
    for pattern in intent['patterns']:
        # Tokenize mỗi từ trong câu
        w = tokenize(pattern)
        # Thêm vào danh sách từ của chúng ta
        all_words.extend(w)
        # Thêm vào cặp xy
        xy.append((w, tag))
    

## Tiền xử lý Dữ liệu Văn bản

In [23]:
# Stem và chuyển mỗi từ thành chữ thường
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# Loại bỏ các từ trùng lặp và sắp xếp
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

66 patterns
12 tags: ['delivery', 'feedback', 'funny', 'goodbye', 'greeting', 'hours', 'items', 'location', 'payments', 'reservation', 'special_request', 'thanks']
196 unique stemmed words: [',', '5', 'ai', 'bao', 'biết', 'biệt', 'buổi', 'bàn', 'bán', 'bâi', 'bạn', 'bản', 'bật', 'bằng', 'bị', 'cay', 'chay', 'chi', 'cho', 'chuyện', 'chuẩn', 'chào', 'chúc', 'chấp', 'chỉ', 'cuối', 'các', 'cách', 'câu', 'có', 'cười', 'cảm', 'cần', 'cầu', 'cọc', 'của', 'cửa', 'dị', 'dịch', 'dụng', 'gian', 'giao', 'giá', 'giúp', 'giới', 'giờ', 'gì', 'góp', 'gần', 'gặp', 'gửi', 'hay', 'hoặc', 'hài', 'hàng', 'hành', 'hãi', 'hôm', 'hơn', 'hước', 'hẹn', 'hồi', 'hữu', 'khen', 'khi', 'khác', 'khách', 'không', 'khỏe', 'kiến', 'kể', 'liên', 'là', 'làm', 'lâu', 'lòng', 'lưu', 'lại', 'lần', 'lễ', 'mastercard', 'menu', 'momo', 'muối', 'muốn', 'món', 'mấi', 'mất', 'mặt', 'một', 'mới', 'mở', 'nay', 'nghe', 'ngài', 'người', 'ngợi', 'nhanh', 'nhiều', 'nhà', 'nhánh', 'nhé', 'nhóm', 'nhất', 'nhận', 'nhật', 'những', 'nài', 'n

##  Tạo Dữ liệu Huấn luyện

In [24]:
# Tạo dữ liệu huấn luyện
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words cho mỗi pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss chỉ cần nhãn lớp, không phải one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

## Định nghĩa Siêu tham số và Kích thước Mô hình

In [25]:
# Siêu tham số
num_epochs = 300
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print("input_size:", input_size)
print("output_size:", output_size)

input_size: 196
output_size: 12


## Tạo Lớp ChatDataset (Dataset tùy chỉnh của PyTorch)

In [26]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # Hỗ trợ lập chỉ mục để dataset[i] có thể được sử dụng để lấy mẫu thứ i
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # Chúng ta có thể gọi len(dataset) để trả về kích thước
    def __len__(self):
        return self.n_samples

## Tạo DataLoader

In [27]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

## Khởi tạo Mô hình, Hàm Mất mát và Trình Tối ưu hóa

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Hàm mất mát và trình tối ưu hóa
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Huấn luyện Mô hình

In [29]:
# Huấn luyện mô hình
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        # labels cần được chuyển đổi sang kiểu LongTensor cho CrossEntropyLoss
        labels = labels.to(dtype=torch.long).to(device)
        
        # Lan truyền tiến (Forward pass)
        outputs = model(words)
        # Nếu y là one-hot, chúng ta phải áp dụng
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Lan truyền ngược và tối ưu hóa (Backward and optimize)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/300], Loss: 0.1104
Epoch [200/300], Loss: 0.0207
Epoch [300/300], Loss: 0.0026


## In Mất mát Cuối cùng và Lưu Mô hình

In [30]:
print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)


final loss: 0.0026
